# `bmorph` Example Workflow Template
This notebook demonstrates how to setup data for and bias correct it through **bmorph**. ``bmorph_exaple_workflow.rst`` explains how this template works.

## Import Packages and Load Data

In [ ]:
%pylab inline
import xarray as xr
import pandas as pd
import bmorph
from bmorph.util import mizuroute_utils as mizutil
from dask.distributed import Client, progress

xr.set_options(display_style='html') # this is just to make viewing our data nicer

In [ ]:
client = Client(threads_per_worker=1, n_workers=1)

In [ ]:
site_to_seg = { site_0_name : site_0_seg, ...} # Input this mapping or read it from a text file before running!

seg_to_site = {seg: site for site, seg in site_to_seg.items()}
ref_sites = list(site_to_seg.keys())
ref_segs = list(site_to_seg.values())

basin_topo = xr.open_dataset('../topologies/basin_topology_file_name.nc').load()

watershed_met = xr.open_dataset('../input/tmin.nc').load()
watershed_met['seasonal_precip'] = xr.open_dataset('../input/prec.nc')['prec'].load().rolling(time=30, min_periods=1).sum()
watershed_met['tmax'] = xr.open_dataset('../input/tmax.nc')['tmax'].load()
watershed_met['hru'] = (watershed_met['hru'] - 1.7e7).astype(np.int32)
watershed_raw = xr.open_mfdataset('../input/first_route*.nc')[['IRFroutedRunoff', 'dlayRunoff', 'reachID']].load()
watershed_raw['seg'] = watershed_raw.isel(time=0)['reachID'].astype(np.int32)

watershed_ref = xr.open_dataset('../input/..').load().rename({'outlet':'site'})[['seg', 'seg_id', 'reference_flow']]

watershed_topo = xr.open_dataset('../topologies/watershed_topology_file_name.nc').load()
watershed_topo = watershed_topo.where(watershed_topo['hru'] < 1.79e7, drop=True)

if 'hru_id2' in basin_topo:
    basin_topo['hru'] = basin_topo['hru_id2']
if 'seg_id' in basin_topo:
    basin_topo['seg'] = basin_topo['seg_id']

## Convert `mizuroute` formatting to `bmorph` formatting

In [ ]:
basin_ref = watershed_ref.sel(site=[r for r in ref_sites])

for site, seg in site_to_seg.items():
    if site in basin_ref['site']:
        basin_ref['seg'].loc[{'site': site}] = seg

# `mizuroute_to_blendmorph` is a utility function that automates
# the preprocessing for bmorph
basin_met_seg = mizutil.mizuroute_to_blendmorph(
    basin_topo, watershed_raw.copy(), basin_ref, watershed_met, 
    fill_method='r2').ffill(dim='seg')

## Apply `bmorph` bias correction

In [ ]:
train_window = pd.date_range('1981-01-01', '1990-12-30')[[0, -1]]
bmorph_window = pd.date_range('1991-01-01', '2005-12-30')[[0, -1]]
reference_window = train_window

interval = pd.DateOffset(years=1)
overlap = 90
#condition_var = 'tmax'
#condition_var = 'seasonal_precip'
condition_var = 'tmin'

conditonal_config = {
    'train_window': train_window,
    'bmorph_window': bmorph_window,
    'reference_window': reference_window,
    'bmorph_interval': interval,
    'bmorph_overlap': overlap,
    'condition_var': condition_var
}

univariate_config = {
    'train_window': train_window,
    'bmorph_window': bmorph_window,
    'reference_window': reference_window,
    'bmorph_interval': interval,
    'bmorph_overlap': overlap,
}

In [ ]:
ibc_u_flows = {}
ibc_u_mults = {}
ibc_c_flows = {}
ibc_c_mults = {}

raw_flows = {}
ref_flows = {}

for site, seg in site_to_seg.items():
    raw_ts = basin_met_seg.sel(seg=seg)['IRFroutedRunoff'].to_series()
    train_ts = basin_met_seg.sel(seg=seg)['IRFroutedRunoff'].to_series()
    obs_ts = basin_met_seg.sel(seg=seg)['up_ref_flow'].to_series()
    cond_var = basin_met_seg.sel(seg=seg)[f'up_{condition_var}'].to_series()
    ref_flows[site] = obs_ts
    raw_flows[site] = raw_ts
    
    ## IBC_U
    ibc_u_flows[site], ibc_u_mults[site] = bmorph.workflows.apply_interval_bmorph(
        raw_ts, train_ts, obs_ts, train_window, bmorph_window, reference_window, interval, overlap)
    
    ## IBC_C
    ibc_c_flows[site], ibc_c_mults[site] = bmorph.workflows.apply_interval_bmorph(
        raw_ts, train_ts, obs_ts, train_window, bmorph_window, reference_window, interval, overlap,
        raw_y=cond_var, train_y=cond_var, obs_y=cond_var)

In [ ]:
mizuroute_exe = # mizuroute designation

unconditioned_totals = {}
conditioned_totals = {}
region = # basin name

unconditioned_totals = bmorph.workflows.run_parallel_scbc(basin_met_seg, client, region, mizuroute_exe, univariate_config)
conditioned_totals = bmorph.workflows.run_parallel_scbc(basin_met_seg, client, region, mizuroute_exe, conditonal_config)
for site, seg in site_to_seg.items():
    unconditioned_totals[site] = unconditioned_totals['IRFroutedRunoff'].sel(seg=seg)
    conditioned_totals[site] = conditioned_totals['IRFroutedRunoff'].sel(seg=seg)

In [ ]:
scbc_c = bmorph.workflows.bmorph_to_dataarray(conditioned_totals, 'scbc_c')
basin_analysis = xr.Dataset(coords={'site': list(site_to_seg.keys()), 'time': scbc_c['time']})
basin_analysis['scbc_c'] = scbc_c
basin_analysis['scbc_u'] = bmorph.workflows.bmorph_to_dataarray(unconditioned_totas, 'scbc_u')
basin_analysis['ibc_u'] = bmorph.workflows.bmorph_to_dataarray(ibc_u_flows, 'ibc_u')
basin_analysis['ibc_c'] = bmorph.workflows.bmorph_to_dataarray(ibc_c_flows, 'ibc_c')
basin_analysis['raw'] = bmorph.workflows.bmorph_to_dataarray(raw_flows, 'raw')
basin_analysis['ref'] = bmorph.workflows.bmorph_to_dataarray(ref_flows, 'ref')
basin_analysis.to_netcdf(f'../output/{region.lower()}_data_processed.nc')